In [1]:
import pandas as pd

In [37]:
users = pd.read_csv('sushi3.udata', sep='\t', names=('uid', 'gender', 'age', 'time', 'old_prefecture', 'old_region', 'old_eastwest', 'prefecture', 'region', 'eastwest', 'same'))

In [57]:
users.head()

,uid,gender,age,time,old_prefecture,old_region,old_eastwest,prefecture,region,eastwest,same
0,6371,0,2,355,0,0,0,0,0,0,0
1,10007,1,1,214,26,6,1,26,6,1,0
2,1777,1,2,363,29,6,1,26,6,1,1
3,3613,0,4,395,40,9,1,26,6,1,1
4,8081,1,1,707,26,6,1,36,8,1,1


In [58]:
users['time'].max()

877

In [38]:
items = pd.read_csv('sushi3.idata', sep='\t', names=('iid', 'name', 'style', 'major', 'minor', 'heaviness', 'frequency', 'price', 'popularity'))

In [16]:
R = pd.read_csv('sushi3b.5000.10.score', sep=' ', header=None)

In [14]:
import numpy as np

In [30]:
triplets = []
for i, line in enumerate(np.array(R)):
    for j, v in enumerate(line):
        if v != -1:
            triplets.append((i, j, v))

In [31]:
triplets = pd.DataFrame(triplets)

In [28]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [32]:
len(triplets)

50000

In [33]:
reader = Reader(rating_scale=(0, 4))
data = Dataset.load_from_df(triplets, reader)

In [52]:
cross_validate(NormalPredictor(), data, cv=5)

{'test_rmse': array([1.67945053, 1.6617848 , 1.67099799, 1.6709339 , 1.66334078]),
 'test_mae': array([1.34219596, 1.31938237, 1.32877073, 1.33617369, 1.32668685]),
 'fit_time': (0.048548221588134766,
  0.04827404022216797,
  0.049672842025756836,
  0.04931998252868652,
  0.04492783546447754),
 'test_time': (0.07823896408081055,
  0.08065009117126465,
  0.07845592498779297,
  0.07151603698730469,
  0.06636595726013184)}

In [53]:
from surprise import BaselineOnly

cross_validate(BaselineOnly(), data, cv=5)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([1.14760298, 1.1434182 , 1.13474427, 1.15332481, 1.15031282]),
 'test_mae': array([0.93734735, 0.9321307 , 0.92515215, 0.9447453 , 0.94228576]),
 'fit_time': (0.047470808029174805,
  0.05024313926696777,
  0.05152177810668945,
  0.0469050407409668,
  0.048692941665649414),
 'test_time': (0.06548500061035156,
  0.0653681755065918,
  0.06401300430297852,
  0.0656578540802002,
  0.05430293083190918)}

In [35]:
from surprise import SVD

In [78]:
cross_validate(SVD(n_factors=10, n_epochs=20), data, cv=5)

{'test_rmse': array([1.13963885, 1.12947456, 1.14059369, 1.13946482, 1.13123579]),
 'test_mae': array([0.92119985, 0.91893264, 0.9252735 , 0.92345343, 0.91644076]),
 'fit_time': (0.9610450267791748,
  1.022212028503418,
  0.9887239933013916,
  0.9414162635803223,
  0.9597580432891846),
 'test_time': (0.07801222801208496,
  0.0704200267791748,
  0.06885313987731934,
  0.06659889221191406,
  0.06396198272705078)}

In [64]:
u_cont = {'time'}
u_cat = set(users.columns) - u_cont
u_cat

{'age',
 'eastwest',
 'gender',
 'old_eastwest',
 'old_prefecture',
 'old_region',
 'prefecture',
 'region',
 'same',
 'uid'}

In [65]:
i_cont = {'heaviness', 'frequency', 'price', 'popularity'}
i_cat = set(items.columns) - i_cont - {'name'}
i_cat

{'iid', 'major', 'minor', 'style'}

In [66]:
Xi = []
Xv = []
y = []
for i, j, v in np.array(triplets):
    indexes = []
    values = []
    
    indexes.extend(list(users.iloc[i][u_cat]))
    values.extend([1] * len(u_cat))
    indexes.extend([1] * len(u_cont))
    values.extend(list(users.iloc[i][u_cont] / 877))
    
    indexes.extend(list(items.iloc[j][i_cat]))
    values.extend([1] * len(i_cat))
    indexes.extend([1] * len(i_cont))
    values.extend(list(items.iloc[j][i_cont]))
    
    assert len(indexes) == len(values)
    Xi.append(indexes)
    Xv.append(values)
    y.append(v)

In [67]:
len(Xi), len(Xv), len(y), len(Xi[0])

(50000, 50000, 50000, 19)

In [70]:
import pickle

with open('sushi.pickle', 'wb') as f:
    # pickle.dump(Xi, f, pickle.HIGHEST_PROTOCOL)
    # pickle.dump(Xv, f, pickle.HIGHEST_PROTOCOL)
    # pickle.dump(y, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(Xi, f, 2)
    pickle.dump(Xv, f, 2)
    pickle.dump(y, f, 2)

In [79]:
users.shape

(5000, 11)

In [80]:
items.shape

(100, 9)